### This file call all predicting methods on GEF2012Com and Hvaler
Use MeasurePerformance.ipynb file to measure performance of produced prediction

In [1]:
library(tidyr)
library(dplyr)
library(lubridate)
library(xts)
library("doParallel")
library("foreach")
source("Lib/PredictRandomForest.R")
source("Lib/PredictDSHW.R")
source("Lib/PredictSemiParametricArima.R")
source("Lib/PredictTBATS.R")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


In [2]:
#Predict for Hvaler
HvalerTrainingFile = "Hvaler/training_set.csv"
HvalerCompleteFile = "Hvaler/imputed_complete.csv"
OutputDir = "Hvaler/Predictions/"
HvalerClasses = c('POSIXct', rep("numeric", 21))
NCores = 8
Zones = paste0("subs.", seq(1, 20))#Only test 1 zone now
Temperatures = c("T01")
Horizons = seq(1, 24)
trainingDf = read.csv(HvalerTrainingFile, stringsAsFactors=FALSE, colClasses=HvalerClasses)
completeDf = read.csv(HvalerCompleteFile, stringsAsFactors=FALSE, colClasses=HvalerClasses)


In [ ]:
#BE CAREFUL: This will take several hours!
predictRandomForest(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, nDataPoints = -1, PlotResult = FALSE)


In [ ]:
#This will also take a lot of time
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=FALSE, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
#This will also take a lot of time
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=TRUE, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
#This will also take a lot of time
registerDoParallel(NCores)
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictSemiParametricArima(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
#This will also take a lot of time
registerDoParallel(NCores)
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictTBATS(OutputDir, trainingDf, completeDf, Zones, Horizons, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
#Predict for GEFCom2012
GEFCom2012TrainingFile = "GEFCom2012/training_set.csv"
GEFCom2012CompleteFile = "GEFCom2012/complete.csv"
OutputDir = "GEFCom2012/Predictions/"
GEFCom2012Classes = c('POSIXct', rep("numeric", 32))
Zones = paste0("zone.", seq(1, 20))
Temperatures = c("T01","T02","T03","T04","T05","T06","T07","T08","T09","T10","T11")
Horizons = seq(1, 24)
trainingDf = read.csv(GEFCom2012TrainingFile, stringsAsFactors=FALSE, colClasses=GEFCom2012Classes)
completeDf = read.csv(GEFCom2012CompleteFile, stringsAsFactors=FALSE, colClasses=GEFCom2012Classes)

In [ ]:
#This will take several hours!
predictRandomForest(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, nDataPoints = -1, PlotResult = FALSE)

In [ ]:
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=FALSE, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=TRUE, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictSemiParametricArima(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
registerDoParallel(NCores)
x = foreach(Zones = Zones, .combine=c) %dopar% 
    predictTBATS(OutputDir, trainingDf, completeDf, Zones, Horizons, PlotResult = FALSE)
stopImplicitCluster()

In [ ]:
install.packages("ranger")

In [ ]:
install.packages("bbemkr")

In [ ]:
install.packages("doParallel")
install.packages("foreach")